In [1]:
%cd ..

/home/wiss/hofherrf/gitRepos/pendulum_2d


In [2]:
import os
import yaml
import torch
from torchvision import utils
from models.sceneRepresentation import Scene
from dataset.dataset import ImageDataset_realData
from util.util import compute_psnr
import matplotlib.pyplot as plt

In [22]:
path_experiment = os.path.join(
    os.path.abspath(''),
    'experiments',
    '2023-01-24',
    'real_world',
    'slidingBlock',
    '10-38-53'
)

In [23]:
path_conf = os.path.join(path_experiment, '.hydra','config.yaml')
with open(path_conf) as f:
    cfg = yaml.safe_load(f)

In [24]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [25]:
model = Scene(**cfg['scene']['background'])

model.add_slidingBlock(
    alpha=torch.tensor(0.),
    p0=torch.ones(2),
    **cfg['scene']['local_representation']
)

path_ckpt = os.path.join(path_experiment, 'ckpt.pth')
model.load_state_dict(torch.load(path_ckpt))

model.use_homography = cfg['homography']['enable']

In [26]:
model.to(device)
print("Moved to device")

Moved to device


In [27]:
data_path = os.path.join(os.path.abspath(''), 'data',cfg['data']['path_data'])

train_data = ImageDataset_realData(
    **cfg['data'],
    max_samples=cfg['data']['samples_train']
)

dataset = ImageDataset_realData(
    **cfg['data'],
    indices=train_data.unused_inds
)

tspan = dataset.t_steps.to(device)
print(f"Tspan eval data: {tspan}")
print(f"Tspan train data: {train_data.t_steps}")

Tspan eval data: tensor([0.0000, 0.0333, 0.1000, 0.1667, 0.2333, 0.3000, 0.3667, 0.4000],
       device='cuda:0')
Tspan train data: tensor([0.0000, 0.0667, 0.1333, 0.2000, 0.2667, 0.3333])


In [28]:
# Render images
model.eval()
model.update_trafo(tspan)
H, W = dataset.get_image_dim()
output = model.render_image(W, H)
ims = output["Image"].cpu()
masks = output['Mask'].cpu()

psnr = compute_psnr(ims, dataset.get_full_images())
norm = torch.norm(torch.eye(3) - model.homography_matrix.cpu())

measured_alpha = 24
rel_error_alpha = torch.abs(torch.rad2deg(model.local_representation.ode.alpha) - measured_alpha) / measured_alpha
print(f"PSNR: {psnr}, Norm diff: {norm}")
print(f"Rel Error alpha: {100*rel_error_alpha:.2f}%")
print(f"{psnr:.2f} & {norm:.2f}")

PSNR: 34.761505126953125, Norm diff: 0.1761433184146881
Rel Error alpha: 3.21%
34.76 & 0.18


In [14]:
path_folder_rendering = os.path.join(path_experiment, 'renderingsRealWorld')
os.makedirs(path_folder_rendering)

# Save individual images
for i in range(len(tspan)):
    path = os.path.join(path_folder_rendering, f"{i+1}_eval.jpg")
    cur_im = ims[i].permute(2, 0, 1)
    utils.save_image(cur_im, path)

    path = os.path.join(path_folder_rendering, f"{i+1}_mask_eval.jpg")
    cur_mask = masks[i]
    utils.save_image(cur_mask, path)

    path = os.path.join(path_folder_rendering, f"{i+1}_gt.jpg")
    cur_im_gt = dataset.get_full_images(i).permute(2, 0, 1)
    utils.save_image(cur_im_gt, path)